# Домашнее задание 2 (5 баллов).

*Все задания ниже имеют равный вес (5/10)*

Код для импорта мы написали за вас (не благодарите, нам не трудно). Дальше код будете писать вы.

[Тут](https://habr.com/ru/companies/ruvds/articles/494720/) шпора по pandas. За основу домашнего задания взят ноутбук [отсюда](https://rutube.ru/video/f884aa6ed5f94120b7304506042fe5bb/) (не подглядывайте!).

In [ ]:
import pandas as pd
import numpy as np

#### Описание данных

Автор д/з - плохой человек, который не стал переводить описание с мотивировкой, что весь DS на английском. Так что описание полей будет на английском:

1. Account ID
- Description: A unique identifier for each social media account in the dataset.
- Type: Integer
- Example: 1, 2, 3, …
2. Username
- Description: The username or handle of the social media account.
- Type: String
- Example: john_doe, tech_guru_22, fitness_freak
3. Platform
- Description: The social media platform the account is using (Instagram, Twitter, Facebook, TikTok, LinkedIn).
- Type: Categorical (String)
- Example: Instagram, Twitter, Facebook, TikTok, LinkedIn
4. Follower Count
- Description: The total number of followers the account has.
- Type: Integer
- Example: 1500, 245000, 78000
5. Posts Per Week
- Description: The average number of posts the account creates per week.
- Type: Integer
- Example: 3, 5, 7
6. Engagement Rate
- Description: The percentage of interactions (likes, comments, shares) relative to the follower count. This is a measure of how engaging the content is.
- Type: Float
- Range: 0.01 to 0.15
- Example: 0.045 (4.5% engagement rate)
7. Ad Spend (USD)
- Description: The monthly amount spent on advertising or promoting posts.
- Type: Float
- Example: 150.75, 850.00, 300.50
8. Conversion Rate
- Description: The percentage of users who take a desired action (e.g., clicking a link, signing up, etc.) after interacting with an ad.
- Type: Float
- Range: 0.01 to 0.05 (1% to 5% conversion rate)
- Example: 0.025 (2.5% conversion rate)
9. Campaign Reach
- Description: The total number of unique users reached by the user’s campaigns in a given month.
- Type: Integer
- Example: 5000, 20000, 15000

#### Задание 0

Подгрузите данные. Да-да, за чтение таблицы баллов не будет))

**Hint**: [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [ ]:
df = ... # ^⨀ᴥ⨀^

In [ ]:
df = pd.read_csv("data.csv")

#### Задание 1

Колонка `Platform` содержит название различных платформ. Давайте представим, что в них есть некоторое отношение порядка. Закодируйте каждую платформу целым числом (от 0 до N) и положите этот "код" в новую колонку `Platform_Code`. Теперь вычислите корреляцию Спирмена между всеми парами колонок в датасете (результатом будет таблица корреляций). В качестве ответа выведите значение корреляции `Platform_Code` с `Engagement Rate`. Можете после вывода числа еще коротко написать, что оно означает (нет, это не оценивается).

**Hint**: [pd.factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html), [pd.DataFrame.select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), [pd.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html).

In [ ]:
df['Platform_Code'] = pd.factorize(df['Platform'])[0]
corrPE = df.select_dtypes(include=['int', 'float']).corr(method='spearman').loc['Platform_Code', 'Engagement Rate']


In [ ]:
corrPE

0.03138169529349812

In [ ]:
if round(corrPE) == 0:
  print('Почти нет зависимости')

Почти нет зависимости


#### Задание 2

Теперь посмотрите на столбец `Follower Count`. В нем какие-то числа. Иногда бывает полезно провести дискретизацию такого признака. Разбейте все значения в столбце на 4 группы: "Low", "Medium", "High", "Very High". Каждая группа включает в себя новые 25% данных. То есть, Low включает в себя 25% самых маленьких значений признака и так далее. Положите значения "Low", "Medium", "High" или "Very High" для каждого сэмпла датасета в новую колонку `Follower_Bin`. Теперь посчитайте среднее значение `Engagement Rate` для каждой категории из `Follower_Bin`. В качестве ответа выведите значение для категории "High".

**Hint**: [pd.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html), [pd.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), [pd.DataFrame.mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html)

In [ ]:

df["Follower_Bin"] = pd.qcut(df["Follower Count"], q=4, labels=["Low", "Medium", "High", "Very High"])
gr_table = df.groupby(by='Follower_Bin')['Engagement Rate']
gr_table.mean()['High']

<ipython-input-7-91446adf91f6>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  gr_table = df.groupby(by='Follower_Bin')['Engagement Rate']


0.08655032

#### Задание 3

Иногда бывает полезно превратить широкую таблицу в длинную (например, для визуализаций сразу нескольких признаков на одной картинке). Да, звучит странно, но именно этим вы сейчас и займетесь. Сделайте новый датафрейм `melted_df`, в который вы поместите каждый сэмпл датасета 6 раз: по одному разу на значение из 'Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate' и 'Campaign Reach'. То есть, вы берете сэмпл из датасета (строку) и превращаете ее в 6 отдельных строк. Каждая отдельная строка в столбце `Metric` имеет имя из предложенного списка 5 признаков, а в столбце `Value` - значение данного сэмпла по этому признаку. Значение `Platform` повторяется в этих 6 строках.

Иначе говоря,

```json
{
    "Account ID": 1,
    "Username": "harrislisa",
    "Platform": "TikTok",
    "Follower Count": 54217,
    "Posts Per Week": 3,
    "Engagement Rate": 0.0986,
    "Ad Spend (USD)": 538.1,
    "Conversion Rate": 0.049,
    "Campaign Reach": 1308,
    "Platform_Code": 0,
    "Follower_Bin": "Low"
}
```

превращается в

```json
{
    "Platform": "TikTok",
    "Metric": "Follower Count",
    "Value": 54217,
},
{
    "Platform": "TikTok",
    "Metric": "Posts Per Week",
    "Value": 3,
}, ...
```

Для каждого уникальной пары значений (`Platform`, `Metric`) посчитайте моду среди всех значений `Value` для этой пары, результат сделайте списком и оставьте только наибольшее. В качестве ответа выведите сумму полученных мод (сумму всех значений в столбце `Value` уже после вычисления мод). Иначе говоря, выведите сумму всех мод значений для всех уникальных пар (`Platform`, `Metric`).

**Hint**: [pd.melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html), [pd.DataFrame.mode](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mode.html), [pd.DataFrameGroupBy.agg](https://pandas.pydata.org/docs/dev/reference/api/pandas.core.groupby.DataFrameGroupBy.agg.html)

In [ ]:
melted_df = pd.melt(df[['Platform', 'Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate', 'Campaign Reach']], id_vars=['Platform'], var_name='Metric', value_name='Value')
res = melted_df.groupby(by=['Platform', 'Metric']).agg({'Value': lambda x: x.mode().max()})
res['Value'].sum()

3100285.4716

#### Задание 4

А теперь хочется посмотреть на самые популярные аккаунты на разных платформах. Для каждой платформы отсортируйте датафрейм по убыванию количества подписчиков (`Follower Count`) - да, без циклов, сразу для всех платформ сделать сортировку, а затем оставьте только первые три записи для каждой платформы - это и будут три самых популярных аккаунта для каждой платформы. В качестве ответа выведите саму таблицу и минимальное значение `Follower Count` в ней.

**Hint**: к *groupby* можно применять функции - это эквивалентно применению функции к каждой "группе" внутри groupby-объекта. Читайте [про применение apply к датафрейму после groupby](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply).

In [ ]:
most_followed = melted_df.groupby(by=['Platform']).apply(lambda x: x.sort_values(by='Value', ascending=False)[:3])
# print(most_followed)
print(np.min(most_followed['Value']))

997512.0


<ipython-input-9-5cc84f13ce85>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  most_followed = melted_df.groupby(by=['Platform']).apply(lambda x: x.sort_values(by='Value', ascending=False)[:3])


#### Задание 5

Хочется посчитать какую-то метрику. Мы хотим посмотреть, на отношение разности суммы подписчиков аккаунтов с высокой и низкой конверсией к суммарному охвату рекламы на каждой платформе. То есть, мы делим аккаунты на две группы: высокая и низка конверсия. Затем мы смотрим на то, на сколько сильно влияние аккаунтов с высокой конверсией по сравнению с аккаунтами с низкой конверсией.

Давайте определим *Conversion Influence* следущим образом:

$$Conversion Influence = \frac{Total Follower\ Count (High) - Total Follower\ Count (Low)}{Total Campaign Reach (High)+Total Campaign Reach (Low)}$$

Считать эту метрику мы будет для каждой `Platform`. В этой формуле High - это значения всех сэмплов датасета, в которых `Conversion Rate` больше медианы, а `Low` - не более медианы. `Total Feature` - это суммарное количество значений `Feature` либо по `High` сэмплам, либо по `Low`.

Чтобы постоянно не пересчитывать, где High. где Low, сделайте новую колонку в датасете `Conversion_Category`. Положите в нее для каждой строки либо High, либо Low.

Выведите платформу с самым большим `Conversion Influence`.

**Hint**: данное задание не про *groupby*, а скорее про [pd.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html). Сделайте сводную таблицу, по которой уже можно посчитать суммы, а затем подставить их в формулы.

In [ ]:
medians = df.groupby(by=['Platform']).agg({'Conversion Rate': 'mean'})
df['Conversion_Category'] = df.apply(lambda x: 'High' if x ['Conversion Rate'] > medians.loc[x['Platform'], 'Conversion Rate'] else 'Low', axis=1)

pt = pd.pivot_table(df, index=['Platform', 'Conversion_Category'] , values=['Campaign Reach', 'Follower Count', 'Conversion Rate'], aggfunc='sum')
pt.reset_index(inplace=True)
high_df = pt[pt['Conversion_Category'] == 'High'].set_index('Platform')
low_df = pt[pt['Conversion_Category'] == 'Low'].set_index('Platform')

ci_df = ((high_df['Follower Count'] - low_df['Follower Count']) / (high_df['Campaign Reach'] + low_df['Campaign Reach'])).reset_index(name='ConversionInfluence')

ci_df.loc[ci_df['ConversionInfluence'].idxmax()]['Platform']


'Twitter'

#### Задание 6

Мы знаем, что вам понравилось считать метрики по формуле. Давайте закрепим этот успех. Теперь для каждой платформы посчитаем, на сколько эффективна реклама в разрезе трех последовательных записей в датасете.

Для каждой платформы отсортируйте записи в порядке убывания `Posts Per Week`. Будто бы аккаунты, которые постят чаще, используют более "активные" стратегии по рекламе. Теперь посчитайте *скользущие суммы с окном 3* по `Campaign Reach` и `Ad Spend (USD)`. Скользящая сумма с окном N - это вы идете по массиву, берете все последовательные тройки записей и суммируете их. Для первых двух записей троек не найдется. Для них скользящее среднее - NaN, что нам не помешает.

Теперь для каждого окна посчитайте

$$Rolling Efficiency Ratio = \frac{Rolling Sum of Campaign Reach}{Rolling Sum of Ad Spend}$$

По сути, для каждого окна вы посчитаете сколько пользователе привлеклось за один доллар, потреченный на рекламу, в данном окне. Понятно, что значений будет столько, сколько окон. Нам интересно максимально значение такой эффективности для каждой платформы.

В качестве ответа выведите название платформы с наибольшей максимальной эффективность и наименьшей (два названия, не одно, не три, ровно два).

**Hint**: окна можно делать через [pd.DataFrame.rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html).

In [ ]:
df_grouped = df.groupby(by=['Platform']).apply(lambda x: x.sort_values('Posts Per Week', ascending=False))
df_grouped['RollingSumofCampaignReach'] = df_grouped['Campaign Reach'].rolling(window=3).sum()
df_grouped['RollingSumofAdSpend'] = df_grouped['Ad Spend (USD)'].rolling(window=3).sum()
df_grouped['RollingEfficiencyRatio'] = df_grouped['RollingSumofCampaignReach']/df_grouped['RollingSumofAdSpend']

print(df_grouped['RollingEfficiencyRatio'].idxmax()[0], df_grouped['RollingEfficiencyRatio'].idxmin()[0])

Facebook Instagram


<ipython-input-69-03c57989dd10>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df.groupby(by=['Platform']).apply(lambda x: x.sort_values('Posts Per Week', ascending=False))


#### Задание 7

Это еще не все прекрасные функции pandas, которые мы хотим вам показать. Теперь вы посчитаете, сколько аккаунтов на каждой платформе одновременно лучшие по `Engagement Rate` и `Conversion Rate`.

Сделайте два отдельных суб-сета. В одном оставьте для каждой платфмормы один топовый аккаунт по `Engagement Rate`, в другом - по `Conversion Rate`. Соедините эти два подмножества по столбцу `Platform` так, что в одно строке есть описание сразу двух аккаунтов-лидеров. Теперь посмотрите равны ли имена аккаунтов в одной строке. Выведите количество строк, в которых названия аккаунтов совпадают.

In [ ]:
df = pd.read_csv('data.csv')
df_gr1 = df.groupby(by=['Platform']).agg({'Engagement Rate': 'max'}).reset_index().rename(columns={'Engagement Rate': 'Leader_ER'})
df = df.merge(df_gr1, on='Platform', how='left')
df_gr2 = df.groupby(by=['Platform']).agg({'Conversion Rate': 'max'}).reset_index().rename(columns={'Conversion Rate': 'Leader_CR'})
df = df.merge(df_gr2, on='Platform', how='left')
df = df[(df['Conversion Rate'] == df['Leader_CR']) & (df['Engagement Rate'] == df['Leader_ER'])]
df['Account ID'].count()

3

#### Задание 8

Давайте теперь что-то попроще сделаем. Например, посчитаем отношение суммарного количества подписчиков на аккаунтах с высокой конверсией к такой же сумме в аккаунтах с низкой конверсией (очевидно, для каждой платформы). По сути, мы просто хотим получить число, которое характеризует, на сколько сильно аккаунты с высокой конверсией "доминируют" над аккаунтами с низкой конверсией в плане количества подписчиков.

Высокой конверсией будем считать конверсию больше средней. Остальное - низкая. Посчитайте суммы подписчиков для каждой платформы, поделите одно на другое и выведите разницу между самым большим значением и самым маленьким, а также платформы, которые соотвутствуют этим значениям.

Используйте магическую команду `%%time`, чтобы замерить, сколько времени ушло на исполнение вашего pandas-скрипта.

In [ ]:
%%time
df = pd.read_csv('data.csv')
medians = df.groupby(by=['Platform']).agg({'Conversion Rate': 'mean'})
df['Conversion_Category'] = df.apply(lambda x: 'High' if x ['Conversion Rate'] > medians.loc[x['Platform'], 'Conversion Rate'] else 'Low', axis=1)

df_high = df[df['Conversion_Category'] == 'High']
df_low = df[df['Conversion_Category'] == 'Low']
df_gr_h = df_high.groupby(by=['Platform']).agg({'Follower Count': 'sum'}).reset_index().rename(columns={'Follower Count': 'FC_high'})
df_gr_1 = df_low. groupby(by=['Platform' ]).agg({'Follower Count': 'sum'}). reset_index().rename(columns={'Follower Count': 'FC_low'})
df_gr = pd.merge(df_gr_h, df_gr_1, how='outer', on='Platform')
df_gr['D'] = df_gr['FC_high'] / df_gr['FC_low' ]
print(df_gr['D'].max() - df_gr['D'].min())
df_gr = df_gr.set_index('Platform')
print(df_gr['D'].idxmax(), df_gr['D'].idxmin())


0.14886262659546368
Twitter Instagram
CPU times: user 184 ms, sys: 0 ns, total: 184 ms
Wall time: 184 ms


#### Задание 9

А теперь решите задание 8 чисто питоном. Никаких функций и методов pandas. Только питоновские циклы. Замерьте время выполнения кода. Наконец, сравните время в задании 8 и 9. Напишите ниже, кто же победил: чистый python и pandas?

**Hint**: Чтобы итерироваться по датафрейму, можно из него сделать генератор через [pd.DataFrame.iterrows](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) или [pd.DataFrame.itertuples](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples). К слову, это не все способы итерироваться по датафрейму.

In [ ]:
%%time

data = list(map(lambda x: x.split(','), open('data.csv').read().split('\n')))
pltf_medians = {}
for row in data[1:-1]:
    if row[2] in pltf_medians.keys():
        pltf_medians[row[2]][0] += float(row[-2])
        pltf_medians[row[2]][1] += 1
    else:
        pltf_medians[row[2]] = [float(row[-2]), 1]
pltf_D = {}
for row in data[1:-1]:
    if row[2] not in pltf_D.keys():
        pltf_D[row[2]] = [0, 0]
    if float(row[-2]) > pltf_medians[row[2]][0] / pltf_medians[row[2]][1]:
        pltf_D[row[2]][0] += int(row[3])
    else:
        pltf_D[row[2]][1] += int(row[3])
max_p = [0, None]
min_p = [float('inf'), None]
for pltf in pltf_D.keys():
    k = pltf_D[pltf][0] / pltf_D[pltf][1]
    if k > max_p[0]:
      max_p = [k, pltf]
    elif k < min_p[0]:
      min_p = [k, pltf]
print(max_p[0] - min_p[0])
print(max_p[1], min_p[1])

0.14886262659546368
Twitter Instagram
CPU times: user 26.6 ms, sys: 0 ns, total: 26.6 ms
Wall time: 26.3 ms


**А победителем является**: чистый питон

Потому что автор много писал эффективного питона и мало панадаса. Вывод: автор после дедлайна идет практиковаться в пандас.

#### Задание 10

Крайне серьезное задание. Отнеситесь к нему соответствующе. В ячейке ниже напишите ваш любимый анекдот или мем (только без баянов, окей?). Можно плохие. Помните, это задание на полный балл. Проверяющий работу ассистент должен улыбнуться.

Если вставляете картинку, то убедитесь, что вы ее не подгружаете локально. А то будет неудобно - потерять балл на этом задании, когда надо было выложить картинку на облако и прокинуть ссылку. И нет, нельзя сюда просто ссылку вставить. Либо ищите, как вставить картинку, либо смешной анекдот. Есть всего два стула - выбирайте...

In [ ]:
# Ярослав Витальевич и падающие березки

![Cat GIF](https://lh3.googleusercontent.com/fife/ALs6j_HcGQc7_f4l64jcHGUQJV4Oembi-mx04AOboNwJoqY2hBaN2_lxdGdcQqL_o5A-R2rL8pluO4pRsbGG6bgRc2M92ZiutVAY-wtkf1qNa5FLGJ54JUYWg-CS2jEdtB9FuiLz2bKO82OJ8sJ-uxJljjuHrotihj5-BaEpHuJqqXhefvtJiSfJKz-KZEAPJR3Leo5-EzFMvS1HycuXQ6vwVxLOttoiFDV_kr5fkkUiE7rreNQ2JGVJrYfXGF_Kv4SKqRYX-qvWLw0w-ERvgyWdx7rPtIRdqD1mgt05xEXZT2Sj1rNQ5cOQTce5j02KP0zSgItKvTDa1_R0DsOyKsqVIaUNdzjoK-dsarRv3chJmlkxXUIvASHz8-b6VKuJXi3ZcV79TZzBAiKCNEAi1ohUXBuwDBTN7Qc1My44E-V1K_37R9ytLJpb3_26knXI2xclusgV4_EQgrJxekf61KhbvsA4Q8eA5zEFbgLlIfrgE1w5SnVkKgpgllri0nC7qoCj9mTM-nF192bJyiNwVVf3z0BI3hp4wvhVCAK4VtDRygKSJ3O98GSmIfpz4DJjipgABvyPqwHUYz5QfXxyom6Eio7T3_pPCno19ioccONGbsg5dsyy0BavTXa5SEJxNKX6MHZqHUBW8DDerPzt1qbs-G6DJPBpO5UAiDtz9uDfxGT7D0_XaEDrC8Y8A2Vr0fejRPGr2DIji8_K3e3MwrAaVyWKu97B6Iex4xkn0eOgbOkmlOitj-DIo2RJHhOrzLPr0hx9sr3ZN-VCEnvZES4LpQnjEIj6iZfcxdDRK_OuFXpEafziUUazJvaDuG-tf9y_xXB2HtiBM_YAQhJkt5H3NRFQKEmmUvxg-FZtBYdYJ4NRWb3OUefxcYOdD2D_TlN0Qr0AvQjM5zSz9CR0B8j3SzYI-nicIhbeeBjfx7ftBgimFVZsSWSG6M9gPs6smWFXCAVGM0k9oX-TZrEWz_NV_CNeX5nzly2sFxulcrrBjkgSMVWHjQ8qfglDreeC95dsCzjznsIWFr5ZQbD3AX785FDw4aL6yVlOBGpdOMkWHudEIfwGi62PVEZJMe4u7ZF1EzrslaRWUH-EQIcTV9LQleEbaA-4rcXON-Llvdmo5fo5SpDEWz4FedC7Oqj4J3iy0yRsh8l3Lwwt5bydhInpEHCElCyxCvGuHLW3LAzEprqUvI31FE3CuRCL_jGtdAPn8x6ow_lfK7IUWI_YXbihUE0kukfNvbJe5TW1gwSYWgYavjn_tiMRG7H-SwK5J3jqbbVQQ3BrFa7J_MNgcEK70zHjnK3gyyVVOZkZ7nNWakg74gk9QwIEhZFg2oagoYXWTEapKa1aK5A_55y-05d5KNE2Ew7jweNxmhGZD73-NS7r257ReyqcvoduCRCsfqaXNg7f5-Dz9GXl1R0SgVGBGgMsQg7VSRpUEVsJDQGImySN9Pi2wniBp7J7beTXoYasjhxgZ-GQ6Y1S8iKfgBvzBdqQHUb2FTDsFaLwsY6_Nukwe66xuPL-yuYVPuL8tT48c9fdQvo2LtO8A3WqxiYdHJ_wP_Gp3dZV8P9BxgHprm9UEA724s3fDlrK1zGbBA-Fj5CYlPDdZurd2-26zu4GhvV9C1_XYKdtjOdfqmhmo6PutizHoe9EV2vkD3vhfSxJRNPaNiT_LVQ7wuphixmAZGDjMSOUfKG0EI4ZQ4OWGJbHQSZecLQ3VI-svk1CkEi--r-nqqZ6vyzCwbLT5rukPtaqNaJhy3IRFmUY=w2424-h1432)